# Day 2 — Data caching & smoke checks

This notebook reproduces Day-2 steps:
1) cache video/audio latents (SVD VAE + AudioLDM2 VAE),
2) run the smoke check over the cached items,
3) quickly inspect shapes and a first frame.

In [ ]:
import platform
from pathlib import Path

import torch
import yaml

BASE = Path('/workspace/jointdit').resolve()
print('Repo base:', BASE)
print('Python:', platform.python_version())
print('PyTorch:', torch.__version__)

In [ ]:
cfg_path = BASE / 'configs/day02_cache.yaml'
with open(cfg_path, 'r') as f:
    cfg = yaml.safe_load(f)
cfg

## Cache latents (train & val)
Uses Makefile targets so the CLI is identical to the README.

In [ ]:
import subprocess


def run(cmd):
    print(f"\n$ {cmd}")
    subprocess.run(cmd, shell=True, check=True, cwd=BASE)

run('make cache-train')
run('make cache-val')

## Smoke check (dataset view)
Prints shapes, frame counts, and first-frame path.

In [ ]:
from data_loader.jointdit_dataset import JointDiTDataset

ds = JointDiTDataset(str(BASE/'data/cache'), split='train')
print('[dataset] len(train)=', len(ds))
for i in range(min(3, len(ds))):
    ex = ds[i]
    v_lat = ex['video_latents']['latents']
    a_lat = ex['audio_latents']['latents']
    print(f"- {i}: frames={ex['frame_count']} v_lat={tuple(v_lat.shape)} a_lat={tuple(a_lat.shape)} clip={ex['clip'] is not None}")
    print('  img_firstframe=', ex['img_firstframe'])

## Visualize first frame (quick sanity)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

if len(ds):
    ex0 = ds[0]
    img = Image.open(ex0['img_firstframe']).convert('RGB')
    plt.figure(figsize=(4,3))
    plt.title('Train[0] first frame')
    plt.imshow(img)
    plt.axis('off')
else:
    print('Dataset is empty — run the cache steps first.')